# P6 : Classifiez automatiquement des biens de consommation

## CLASSIFICATION

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from os import listdir
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_auc_score, roc_curve
from glob import glob

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D 
from tensorflow.keras.layers import GlobalAveragePooling1D 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import RandomFlip
from tensorflow.keras.layers import RandomRotation
from tensorflow.keras.layers import RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet import ResNet152, preprocess_input

2023-05-18 17:47:22.084502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## LES DONNÉES

In [3]:
import pandas as pd

# Chargement du jeu de données
df = pd.read_csv('flipkart_com-ecommerce_sample_1050.csv')
# Ajout variable images
path_images = './P6_Bis/Images/'
# Ajout de nouvelles variables 
df['label_name'] = df['product_category_tree'].apply(lambda x: x.split('>>')[0][2:].strip().replace('&', 'and'))
df['image_path'] = path_images + df['label_name'] + '/' + df['image']
df = df[['image', 'label_name']]

# Afficher le résultat
df = pd.DataFrame(df) 


In [4]:

df.head()

,image,label_name
0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590.jpg,Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74.jpg,Baby Care
3,d4684dcdc759dd9cdf41504698d737d8.jpg,Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7.jpg,Home Furnishing


In [5]:
df.shape

(1050, 2)

## ÉTAPE 0 : AJOUT DES LABELS AUX IMAGES

In [5]:
import os
import pandas as pd
import shutil

# Chemin du dossier contenant les images
image_folder = '/Users/danongohou/Desktop/P6_Bis/Images/'

# Charger le DataFrame
df = pd.DataFrame(df)

# Créer un nouveau dossier pour les images avec les labels
new_folder = '/Users/danongohou/Desktop/P6_Bis/NewImages2/'
os.makedirs(new_folder, exist_ok=True)

# Parcourir le DataFrame et traiter chaque ligne
for index, row in df.iterrows():
    # Obtenir le nom de l'image et le label
    image_name = row['image']
    label_name = row['label_name']
    
    # Chemin d'origine de l'image
    original_image_path = os.path.join(image_folder, image_name)
    
    # Nouveau chemin pour l'image avec le label
    new_image_path = os.path.join(new_folder, f"{label_name}_{image_name}")
    
    # Copier l'image vers le nouveau dossier avec le label ajouté
    shutil.copyfile(original_image_path, new_image_path)

# Afficher un message de confirmation
print("Les images ont été copiées avec succès et les labels ont été ajoutés.")


Les images ont été copiées avec succès et les labels ont été ajoutés.


## ÉTAPE 1 :  DIVISION DES DONNÉES 

In [6]:
import os
import random
from PIL import Image
from sklearn.model_selection import train_test_split

# Chemin du dossier contenant les images
image_folder = '/Users/danongohou/Desktop/P6_Bis/NewImages2/'

# Liste des noms de fichiers des images
image_files = os.listdir(image_folder)

# Diviser les données en données d'entraînement et de test
train_files, test_files = train_test_split(image_files, test_size=0.2)


# Vérifier si les dossiers existent, sinon les créer
if not os.path.exists('train_set2'):
    os.makedirs('train_set2')
if not os.path.exists('test_set2'):
    os.makedirs('test_set2')

# Liste des noms de fichiers des images
image_files = os.listdir(image_folder)

# Diviser les données en données d'entraînement et de test
train_files, test_files = train_test_split(image_files, test_size=0.2)

# Sauvegarder les images d'entraînement
for filename in train_files:
    source_path = os.path.join(image_folder, filename)
    destination_path = os.path.join('train_set2', filename)
    image = Image.open(source_path)
    image.save(destination_path)

# Sauvegarder les images de test
for filename in test_files:
    source_path = os.path.join(image_folder, filename)
    destination_path = os.path.join('test_set2', filename)
    image = Image.open(source_path)
    image.save(destination_path)

    
train_folder = './train_set2'  # Dossier d'entraînement contenant les images
test_folder = './test_set2'    # Dossier de test contenant les images


# Compter le nombre d'images dans le dossier d'entraînement
train_image_names = os.listdir(train_folder)
num_train_images = len(train_image_names)

# Compter le nombre d'images dans le dossier de test
test_image_names = os.listdir(test_folder)
num_test_images = len(test_image_names)

print("Nombre d'images dans le dossier d'entraînement : ", num_train_images)
print("Nombre d'images dans le dossier de test : ", num_test_images)

/Users/danongohou/opt/anaconda3/envs/danon/lib/python3.9/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Nombre d'images dans le dossier d'entraînement :  840
Nombre d'images dans le dossier de test :  210


In [6]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Define the list of labels
list_labels = ['Computers', 'Baby Care', 'Beauty and Personal Care',
               'Home Furnishing', 'Kitchen and Dining',
               'Home Decor and Festive Needs', 'Watches']

# Define the path to the train and test images
path = '/Users/danongohou/Desktop/P6_Bis/train_set2/'
path_test = '/Users/danongohou/Desktop/P6_Bis/test_set2/'

# Function to extract label name from image path
def get_label_name(photo):
    for label in list_labels:
        if label in photo:
            return label
    return "no_name"

# Create a DataFrame for the train set
data_train = pd.DataFrame()
data_train["image_path"] = [os.path.join(path, file) for file in os.listdir(path)]
data_train["label_name"] = data_train["image_path"].apply(get_label_name)
label_encoder = LabelEncoder()
data_train["label"] = label_encoder.fit_transform(data_train["label_name"])

# Create a DataFrame for the test set
data_test = pd.DataFrame()
data_test["image_path"] = [os.path.join(path_test, file) for file in os.listdir(path_test)]
data_test["label_name"] = data_test["image_path"].apply(get_label_name)
data_test["label"] = label_encoder.transform(data_test["label_name"])

In [7]:
# Print the resulting train and test DataFrames
data_train.head(5)

,image_path,label_name,label
0,/Users/danongohou/Desktop/P6_Bis/train_set2/Wa...,Watches,6
1,/Users/danongohou/Desktop/P6_Bis/train_set2/Co...,Computers,2
2,/Users/danongohou/Desktop/P6_Bis/train_set2/Be...,Beauty and Personal Care,1
3,/Users/danongohou/Desktop/P6_Bis/train_set2/Co...,Computers,2
4,/Users/danongohou/Desktop/P6_Bis/train_set2/Ki...,Kitchen and Dining,5


In [8]:
print('0: Baby Care')
print('1: Beauty and Personal Care')
print('2: Computers')
print('3: Home Decor and Festive Needs')
print('4: Home Furnishing')
print('5: Kitchen and Dining')
print('6: Watches')


0: Baby Care
1: Beauty and Personal Care
2: Computers
3: Home Decor and Festive Needs
4: Home Furnishing
5: Kitchen and Dining
6: Watches


## ÉTAPE 2 : ImageDataGenerator = le générateur d'images

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Pour mélanger les images, classées initalement par classe
data_train = data_train.sample(frac=1, random_state=42).reset_index(drop=True)

batch_size = 32

def data_flow_fct(data, datagen, data_type=None) :
    data_flow = datagen.flow_from_dataframe(data, directory='',
                                x_col='image_path', y_col='label_name',
                                weight_col=None, target_size=(224, 224),
                                classes=None, class_mode='categorical',
                                batch_size=batch_size, shuffle=True, seed=42,
                                subset=data_type
                                )
    return data_flow


datagen_train = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.15,
    preprocessing_function=preprocess_input)

train_flow = data_flow_fct(data_train, datagen_train, data_type='training')
val_flow = data_flow_fct(data_train, datagen_train, data_type='validation')

datagen_test = ImageDataGenerator(
    validation_split=0,
    preprocessing_function=preprocess_input)

test_flow = data_flow_fct(data_test, datagen_test, data_type=None)


Found 714 validated image filenames belonging to 7 classes.
Found 126 validated image filenames belonging to 7 classes.
Found 210 validated image filenames belonging to 7 classes.


## ÉTAPE 3 : CRÉATION MODÈLE

In [10]:
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

def create_model_fct():
    
    model0 = ResNet152(include_top=False, weights="imagenet", 
                       input_shape=(224, 224, 3))
    for layer in model0.layers:
        layer.trainable = False
    x = model0.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(224, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    predictions = Dense(7, activation='softmax')(x)
    model = Model(inputs=model0.input, outputs=predictions)
    model.compile(loss="categorical_crossentropy", 
                  optimizer='rmsprop', 
                  metrics=["accuracy"])
    print(model.summary())
    return model




In [11]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D 
from tensorflow.keras.layers import GlobalAveragePooling1D 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


# Création du modèle
with tf.device('/gpu:0'): 
    model2 = create_model_fct()

    
# Création du callback
model2_save_path = "./model2_best_weights.h5"
checkpoint = ModelCheckpoint(model2_save_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
callbacks_list = [checkpoint, es]


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block6_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block6_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block6_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_3_conv (Conv2D)   (None, 28, 28, 512)  66048       ['conv3_block6_2_relu[0][0]']    
                                                                                                  
 conv3_block6_3_bn (BatchNormal  (None, 28, 28, 512)  2048       ['conv3_block6_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block1_0_conv (Conv2D)   (None, 14, 14, 1024  525312      ['conv3_block8_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block1_3_conv (Conv2D)   (None, 14, 14, 1024  263168      ['conv4_block1_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block1_0_bn (BatchNormal  (None, 14, 14, 1024  4096       ['conv4_block1_0_conv[0][0]']    
 ization)                       )                                                                 
          

...

                                                                                                  
 conv4_block28_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block27_out[0][0]']      
                                                                                                  
 conv4_block28_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block28_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block28_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block28_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block28_2_conv (Conv2D)  (None, 14, 14, 256)  590080      ['conv4_block28_1_relu[0][0]']   
          

                                )                                                                 
                                                                                                  
 conv4_block31_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block30_out[0][0]']      
                                                                                                  
 conv4_block31_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block31_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block31_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block31_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_blo

 conv4_block33_out (Activation)  (None, 14, 14, 1024  0          ['conv4_block33_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv4_block34_1_conv (Conv2D)  (None, 14, 14, 256)  262400      ['conv4_block33_out[0][0]']      
                                                                                                  
 conv4_block34_1_bn (BatchNorma  (None, 14, 14, 256)  1024       ['conv4_block34_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_1_relu (Activati  (None, 14, 14, 256)  0          ['conv4_block34_1_bn[0][0]']     
 on)                                                                                              
          

                                                                                                  
 conv4_block36_out (Activation)  (None, 14, 14, 1024  0          ['conv4_block36_add[0][0]']      
                                )                                                                 
                                                                                                  
 conv5_block1_1_conv (Conv2D)   (None, 7, 7, 512)    524800      ['conv4_block36_out[0][0]']      
                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)       

                                                                  'conv5_block3_3_bn[0][0]']      
                                                                                                  
 conv5_block3_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block3_add[0][0]']       
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['conv5_block3_out[0][0]']       
 alAveragePooling2D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 224)          458976      ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
 dropout (

In [12]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## ÉTAPE 4 : ENTRAINEMENT DU MODÈLE

In [13]:
with tf.device('/gpu:0'): 
    history = model2.fit(train_flow, 
                        validation_data=val_flow,
                        batch_size=batch_size, 
                        epochs=30, 
                        callbacks=callbacks_list, 
                        verbose=1)
    
    

Epoch 1/30


2023-05-18 17:51:03.344042: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


23/23 [==============================] - ETA: 0s - loss: 1.6316 - accuracy: 0.5924

/Users/danongohou/opt/anaconda3/envs/danon/lib/python3.9/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
2023-05-18 17:53:20.651531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_loss improved from inf to 0.54222, saving model to ./model2_best_weights.h5
23/23 [==============================] - 164s 7s/step - loss: 1.6316 - accuracy: 0.5924 - val_loss: 0.5422 - val_accuracy: 0.8175
Epoch 2/30
23/23 [==============================] - ETA: 0s - loss: 0.6788 - accuracy: 0.7731
Epoch 2: val_loss did not improve from 0.54222
23/23 [==============================] - 149s 6s/step - loss: 0.6788 - accuracy: 0.7731 - val_loss: 0.5539 - val_accuracy: 0.7778
Epoch 3/30
23/23 [==============================] - ETA: 0s - loss: 0.5437 - accuracy: 0.8193
Epoch 3: val_loss improved from 0.54222 to 0.53163, saving model to ./model2_best_weights.h5
23/23 [==============================] - 155s 7s/step - loss: 0.5437 - accuracy: 0.8193 - val_loss: 0.5316 - val_accuracy: 0.8175
Epoch 4/30
23/23 [==============================] - ETA: 0s - loss: 0.5071 - accuracy: 0.8375
Epoch 4: val_loss did not improve from 0.53163
23/23 [==============================] - 152s 7s/st

## ÉTAPE 5 : ÉVALUATION DU MODÈLE

In [14]:
# Score de l'epoch optimal
model2.load_weights(model2_save_path)

loss, accuracy = model2.evaluate(train_flow, verbose=False) 
print("Training Accuracy :  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(val_flow, verbose=False)
print("Validation Accuracy :  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(test_flow, verbose=False)
print("Test Accuracy       :  {:.4f}".format(accuracy))


2023-05-18 18:38:42.262786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Training Accuracy :  0.9692


2023-05-18 18:40:52.096265: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Validation Accuracy :  0.8333


2023-05-18 18:41:20.366982: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test Accuracy       :  0.8810
